In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'strings_to_numbers'
AUTHOR = 'will'

In [72]:
task = Task(
    id=TASK_ID,
    category="Strings",
    description="Convert strings with different formats to numbers",
    plan=[
    ],
    sample_input={
        "numbers": [
            {"format": "comma_sep", "value": "12,337,800"},
            {"format": "under_sep", "value": "80_999"}
        ]
    },
    sample_output=[12337800, 80999],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [73]:
sql = replace(prototype,
    language='sql',
    source='''SELECT 
  CAST((CASE format
    WHEN "comma_sep" THEN REPLACE(value, ",", "")
    WHEN "under_sep" THEN REPLACE(value, "_", "")
    END) AS integer)
FROM numbers''').load_plan()
sql.execute(task)
sql.save()

In [71]:
datalog = replace(prototype,
    language='datalog',
    source='''.decl clean(format:symbol, inp:symbol, i:number, outp:symbol) 
clean(format, inp, 0, "") :- numbers(format, inp).
clean(format, inp, i+1, outp) :-
  clean(format, inp, i, outp_rec),
  i <= strlen(inp),
  chr = substr(inp, i, 1),
  ((format = "comma_sep", sep = ",");
   (format = "under_sep", sep = "_")),
  ((chr  = sep, outp = outp_rec);
   (chr != sep, outp = cat(outp_rec, chr))).

strings_to_numbers(n) :-
  clean(format, inp, strlen(inp), outp),
  n = to_number(outp).''').load_plan()
datalog.execute(task)
datalog.save()

In [70]:
pandas = replace(prototype,
    language='python-pandas',
    source='''def strings_to_numbers(numbers):
    def convert(row):
      sep = "," if row.format == 'comma_sep' else "_"
      return int(row.value.replace(sep, ""))
    return numbers.apply(convert, axis=1).tolist()''').load_plan()
pandas.execute(task)
pandas.save()

In [69]:
imperative = replace(prototype,
    language='python-imperative',
    source='''def strings_to_numbers(numbers):
    output = []
    for row in numbers:
      if row["format"] == 'comma_sep':
        sep = ","
      else:
        sep = "_"
      output.append(int(row["value"].replace(sep, "")))
    return output''').load_plan()
imperative.execute(task)
imperative.save()

In [68]:
functional = replace(prototype,
    language='python-functional',
    source='''def strings_to_numbers(numbers):
    return [
      int(row["value"].replace(
        "," if row["format"] == "comma_sep" else "_", ""))
      for row in numbers        
    ]''').load_plan()
functional.execute(task)
functional.save()